# **Modélisation & IA | DecisionTreeRegressor**
---

## **Etape 0️⃣ : Chargement du dataset :**

In [1]:
import pandas as pd
df=pd.read_csv('data5.csv')
print(df.columns)

Index(['date', 'city', 'profil_cotier', 'température', 'humidity',
       'vitesse_vent', 'precipitation', 'temp_lag1', 'humid_lag1',
       'vitesse_vent_lag1', 'precip_lag1', 'temp_lag3', 'humid_lag3',
       'vitesse_vent_lag3', 'precip_lag3', 'temp_lag7', 'humid_lag7',
       'vitesse_vent_lag7', 'precip_lag7', 'mois', 'Weekend', 'Jours Fériés',
       'saison', 'Vacances Scolaires', 'Ramadan', 'Nouvel An',
       'Indice de Vague de Chaleur', 'Indice de Vague de Froid ',
       'Indice de Pluie Intense', 'Indice de Tempête', 'Indice de sécheresse',
       'boissons fraiches', 'boissons chaudes', 'snacks sucrés',
       'snacks salés', 'produits laitiers frais', 'produits de jardinage',
       'ustensiles jetables', 'crème solaire', 'équipements d urgence',
       'soins hygiene', 'soins hydratants', 'Charbon',
       'produits anti_moustiques'],
      dtype='object')


## **Etape 1️⃣ : Séparation X (features) / y (cibles)**

In [2]:
#variable cible == les produits : 
targets = [
    'boissons fraiches', 'boissons chaudes', 'snacks sucrés',
    'snacks salés', 'produits laitiers frais', 'produits de jardinage',
    'ustensiles jetables', 'crème solaire', 'équipements d urgence',
    'soins hygiene', 'soins hydratants', 'Charbon',
    'produits anti_moustiques'
]
y = df[targets]



In [3]:
#Features (X) :
drop_cols = ['date'] + targets
X = df.drop(columns=drop_cols)


In [4]:
print(type(X))
print(X.shape , y.shape)

<class 'pandas.core.frame.DataFrame'>
(43769, 30) (43769, 13)


## **Pas besoin de normalisation / standardisation**
**Contrairement à des modèles comme la régression linéaire, le SVM ou les réseaux de neurones :**
* **Les arbres de décision ne sont pas sensibles à l’échelle des variables.**
* **Les features peuvent avoir des ordres de grandeur très différents (poids en kg vs prix en euros), l'arbre s'adapte.**

## **Etape 2️⃣ : Encodage des variables catégorielles**

### **One-Hot Encoding (méthode classique avec pd.get_dummies())<br>**
**C’est la méthode la plus simple et adaptée pour les arbres de décision, car :**

* **Elle ne crée pas d'ordre artificiel entre les catégories (ce que ferait LabelEncoder à tort).**

* **Compatible avec DecisionTreeRegressor, RandomForest, GradientBoosting, etc.**

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# Colonnes catégorielles
cat_cols = ['city', 'profil_cotier', 'saison']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)
    ],remainder='passthrough' # garde les colonnes restantes
)

In [5]:
# Transformer les features
X_prepared = preprocessor.fit_transform(X)

# Vérification :
print(X_prepared.shape)


(43769, 55)


## **Bibliothèques nécessaires**

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y, test_size=0.2, random_state=42) # 20% test / 80% train

print(X_train.shape, X_test.shape)

(35015, 55) (8754, 55)


In [12]:
# Modèle de base (sans réglage d'hyperparamètres)
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [15]:
y_pred_train = tree_model.predict(X_train)
y_pred_test = tree_model.predict(X_test)


In [16]:
print("R² Train :", r2_score(y_train, y_pred_train))
print("R² Test  :", r2_score(y_test, y_pred_test))
print("MSE Test :", mean_squared_error(y_test, y_pred_test))

R² Train : 1.0
R² Test  : 0.5781388369745025
MSE Test : 2.7124127871214916


| Jeu       | R²         |
| --------- | ---------- |
| **Train** |  **1.00** |  
| **Test**  |  **0.578** |

➡ **l'arbre apprend parfaitement le train, mais échoue à généraliser : c’est typiquement du OVERFITTING.**


| Métrique     | Régression Linéaire | Arbre de Décision |
|-------------|---------------------|-------------------|
| **R² Test**  |  **0.712**         |  0.578           |
| **MSE Test** |  2.504             |  **2.712**       |

➡ **MSE plus bas avec l'arbre : l’arbre capte mieux certaines relations locales ou non-linéaires, ce qui diminue l’erreur quadratique moyenne.**

➡**R² plus bas : il explique moins de variance globale que la régression linéaire.** 
> ### **L'arbre est plus précis (MSE), mais moins généralisable (R²).**
> * **Un DecisionTreeRegressor par défaut est souvent trop profond, ce qui peut le rendre très bon sur l'entraînement, mais plus faible sur le test**<br>
> * **Il faut le régulariser via des hyperparamètres.**


## **Etape 3️⃣ : Réduction du surapprentissage**

# **Automatiser l’optimisation des hyperparamètres avec GridSearchCV**


## **Les hyperparamètres de l’arbre de décision**


### 1. **`max_depth` (profondeur maximale de l’arbre)**

* **Définition** : Nombre maximum de niveaux (ou "étages") que l’arbre peut atteindre.
* **But** : Contrôler la **complexité** du modèle. Plus la profondeur est grande, plus l’arbre peut s’adapter aux données… jusqu’à l’**overfitting**.


---

### 2. **`min_samples_split` (nombre min. d’échantillons pour scinder un nœud)**

* **Définition** : Nombre minimum d’observations nécessaires pour qu’un nœud soit divisé en deux sous-nœuds.
* **But** : Éviter de splitter un nœud sur des échantillons trop petits → réduit le **bruit**.


---

### 3. **`min_samples_leaf` (taille minimale d’une feuille)**

* **Définition** : Nombre minimal d’échantillons qu'une **feuille** (nœud terminal) doit contenir.
* **But** : Empêche l’arbre de créer des feuilles trop spécifiques avec très peu de données.
* **Conséquence :**

    * `min_samples_leaf=1` (valeur par défaut) → risque de créer des règles trop spécifiques, donc surajustement.
    * `min_samples_leaf=4` → compromis entre précision et robustesse.

---

In [44]:
from sklearn.model_selection import GridSearchCV

In [48]:
model = DecisionTreeRegressor(random_state=42)

# Grille de recherche des hyperparamètres
param_grid = {
    'max_depth': [3,4, 5,6, 7,8, 9,10, 11,12,13,14,15,16,17,18,19,20],
    'min_samples_split': [2,3,4, 5,6,7,8,9, 10,11,12,13,14,15,16,17,18,19, 20],
    'min_samples_leaf': [4, 5,6,7,8,9, 10,11,12,13,14,15,16,17,18,19,20]
}

# Configuration de la recherche avec validation croisée
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='r2',
    cv=5,                    # 5-fold cross-validation
    n_jobs=-1,               # Utilise tous les cœurs CPU disponibles
    verbose=1
)

# Lancer la recherche
grid_search.fit(X_train, y_train)

# Meilleurs paramètres
print("Meilleurs hyperparamètres :", grid_search.best_params_)

# Prédiction avec le meilleur modèle trouvé
best_model = grid_search.best_estimator_
y_pred_test = best_model.predict(X_test)
y_pred_train = tree_model.predict(X_train)

# Évaluation
print("R² Train :", r2_score(y_train, y_pred_train))
print("R² test :", r2_score(y_test, y_pred_test))
print("MSE test :", mean_squared_error(y_test, y_pred_test))

Fitting 5 folds for each of 5814 candidates, totalling 29070 fits
Meilleurs hyperparamètres : {'max_depth': 14, 'min_samples_leaf': 4, 'min_samples_split': 18}
R² Train : 0.7287664347612858
R² test : 0.754588323260129
MSE test : 1.683506948557226


### **L’ajustement automatique par GridSearchCV nous a permis de trouver la bonne complexité d’arbre (max_depth=14) tout en régularisant avec min_samples_leaf=4 et min_samples_split=18.**
> ### Très bon équilibre biais/variance :
>* **Le modèle n’est pas surappris (train ≈ test),**

>* **Et il généralise mieux que tous les précédents (y compris régression + interactions).**

| Hyperparamètre         | Rôle principal                               | Impact dans ton modèle                                           |
| ---------------------- | -------------------------------------------- | ---------------------------------------------------------------- |
| `max_depth=14`         | Limite la profondeur de l’arbre              | Empêche le surapprentissage tout en gardant une bonne complexité |
| `min_samples_split=18` | Évite les divisions sur peu d’exemples       | Réduit le bruit et améliore la stabilité                         |
| `min_samples_leaf=4`   | Importe une taille minimum dans les feuilles | Évite des prédictions basées sur très peu de données             |




In [8]:
#relancer le modele : 
tree_model = DecisionTreeRegressor(
    max_depth=14,              # Limite la complexité de l'arbre
    min_samples_split=18,     # Minimum d'échantillons pour un split
    min_samples_leaf=4,       # Minimum d'échantillons dans une feuille
    random_state=42
)
tree_model.fit(X_train, y_train)
y_pred_train = tree_model.predict(X_train)
y_pred_test = tree_model.predict(X_test)

print("R² Train :", r2_score(y_train, y_pred_train))
print("R² Test  :", r2_score(y_test, y_pred_test))
print("MSE Test :", mean_squared_error(y_test, y_pred_test))


R² Train : 0.7895185994458044
R² Test  : 0.754588323260129
MSE Test : 1.683506948557226


## **Observation :**
| Version du modèle      | R² Train  | R² Test |
| ---------------------- | --------- | ------- |
| Avec `GridSearchCV`    | \~0.729   | \~0.755 |
| En relançant à la main | **0.790** | \~0.755 |


---

### **Pourquoi les résultats se sont améliorés après avoir relancé le modèle ?**

Lors de l’étape d’optimisation des hyperparamètres, la recherche a été effectuée avec `GridSearchCV`, qui repose sur une **validation croisée** : le modèle est alors entraîné sur une portion ( 80 %) du jeu d’entraînement à chaque itération, puis validé sur les 20 % restants. Cela permet une évaluation fiable, mais **le modèle n’est jamais entraîné sur 100 % des données**.

En revanche, après sélection des meilleurs paramètres, une relance du modèle via :

```python
tree_model = DecisionTreeRegressor(...)
tree_model.fit(X_train, y_train)
```

entraîne le modèle **sur 100 % du jeu d’entraînement**, ce qui **explique l’augmentation naturelle du R² sur les données d’entraînement**. Cette hausse est attendue, tant que les performances sur les données de test restent stables ou s'améliorent.

---

Le modèle d’arbre de décision final, entraîné avec les meilleurs hyperparamètres (`max_depth = 14`, `min_samples_split = 18`, `min_samples_leaf = 4`), a produit les résultats suivants :

* **R² Train** : 0.7895
* **R² Test** : 0.7546
* **MSE Test** : 1.68



#### Ces performances montrent que le modèle parvient à **expliquer environ 79 % de la variance** dans les données d'entraînement, tout en conservant une capacité de généralisation élevée avec **75 % de variance expliquée sur les données de test**. L'écart entre les deux scores est faible (environ 3,5 %), ce qui est **un excellent indicateur de stabilité**.

> Un écart faible entre R² train et test indique que le modèle n'est **pas en situation de surapprentissage** (overfitting). Il a appris les relations sous-jacentes dans les données sans les mémoriser.






In [54]:
import numpy as np
# 1. Convertir y_pred en DataFrame si besoin
if isinstance(y_pred_test, np.ndarray):
    y_pred_test = pd.DataFrame(y_pred_test, columns=y_test.columns)

# 2. Concaténer les données
df_long = pd.DataFrame()
for col in y_test.columns:
    temp_df = pd.DataFrame({
        'product': [col] * len(y_test),
        'y_true': y_test[col].values,
        'y_pred': y_pred_test[col].values
    })
    df_long = pd.concat([df_long, temp_df], ignore_index=True)

# 3. Calculer les métriques
metrics = []
for col in y_test.columns:
    r2 = r2_score(y_test[col], y_pred_test[col])
    mse = mean_squared_error(y_test[col], y_pred_test[col])
    metrics.append({'product': col, 'R2': r2, 'MSE': mse})

df_metrics = pd.DataFrame(metrics)

# 4. Fusion finale
df_final = df_long.merge(df_metrics, on='product')

print(df_final.isnull().sum())

product    0
y_true     0
y_pred     0
R2         0
MSE        0
dtype: int64


In [55]:
df_summary = df_final.groupby('product')[['R2', 'MSE']].first().reset_index()
print(df_summary)


                     product        R2       MSE
0                    Charbon  0.880459  0.787541
1           boissons chaudes  0.952159  1.520904
2          boissons fraiches  0.952661  0.921426
3              crème solaire  0.904461  0.677967
4   produits anti_moustiques  0.886394  3.125190
5      produits de jardinage  0.851898  0.648169
6    produits laitiers frais  0.786160  3.234417
7               snacks salés  0.582892  2.892434
8              snacks sucrés  0.740276  3.085017
9           soins hydratants  0.241944  0.832377
10             soins hygiene  0.782411  0.637758
11       ustensiles jetables  0.959901  2.214946
12     équipements d urgence  0.288033  1.307444


## **Comparaison des performances par produit : Arbre de Décision vs Régression Linéaire**
| Produit                   | R² (Arbre) | MSE (Arbre) | R² (RL) | MSE (RL) | Modèle Meilleur  |
| ------------------------- | ---------- | ----------- | ------- | -------- | ---------------- |
| **Charbon**               | 0.880      | 0.788       | 0.848   | 1.004    |  **Arbre**         |
| **Boissons chaudes**      | 0.952      | 1.521       | 0.920   | 2.535    |  **Arbre**         |
| **Boissons fraîches**     | 0.953      | 0.921       | 0.783   | 4.229    |  **Arbre**         |
| **Crème solaire**         | 0.904      | 0.678       | 0.880   | 0.854    |  **Arbre**         |
| **Anti-moustiques**       | 0.886      | 3.125       | 0.893   | 2.949    |  **RL**            |
| **Produits de jardinage** | 0.852      | 0.648       | 0.718   | 1.235    |  **Arbre**         |
| **Produits laitiers**     | 0.786      | 3.234       | 0.771   | 3.467    |  **Arbre**         |
| **Snacks salés**          | 0.583      | 2.892       | 0.535   | 3.223    |  **Arbre**         |
| **Snacks sucrés**         | 0.740      | 3.085       | 0.739   | 3.101    |  **Arbre (léger)** |
| **Soins hydratants**      | 0.242      | 0.832       | 0.205   | 0.873    |  **Arbre**         |
| **Soins hygiène**         | 0.782      | 0.638       | 0.772   | 0.668    |  **Arbre**         |
| **Ustensiles jetables**   | 0.960      | 2.215       | 0.870   | 7.178    |  **Arbre**         |
| **Équipements d’urgence** | 0.288      | 1.307       | 0.325   | 1.239    |  **RL**            |



###  **Interprétation générale** :

*  **L’arbre de décision surpasse la régression linéaire** sur **11 produits sur 13**, en termes de R² **et** de MSE.
*  **Deux produits** où la régression linéaire performe légèrement mieux :

    * **Produits anti-moustiques** : RL a un R² plus élevé et une MSE plus faible.
    * **Équipements d’urgence** : légère supériorité de RL sur le R² et MSE.





# **Doit-on répéter les mêmes étapes que dans RL pour un modèle arbre pour l'améliorer ?**
| Étape                                       | Régression Linéaire   | Modèle Arbre de décision                                         | Faut-il répéter ?                      |
| ------------------------------------------- | --------------------- | ---------------------------------------------------------------- | -------------------------------------- |
| Analyse de multicolinéarité                 | Important (VIF)       | **Peu utile**                                                    | Non, les arbres gèrent la corrélation  |
| Vérification de linéarité                   | Essentielle (résidus) | Pas nécessaire                                                   | Non                                    |
| Variables d’interaction manuelles           | Utile                 | Généralement inutile, car l’arbre capture interactions           | Non, sauf cas particuliers             |
| Transformations polynomiales/logarithmiques | Souvent nécessaire    | Pas nécessaire                                                   | Non                                    |
|
| Sélection de variables                      | SelectKBest           | Moins critique, arbres peuvent gérer cela                        | Non (mais possible via importance)     |
| Régularisation (Lasso, Ridge)               | Oui                   | Différente : paramètres d'arbre (profondeur, min\_samples\_leaf) | Oui, mais avec hyperparamètres d’arbre |
| Correction post-modèle                      | Ajustements manuels   | Souvent moins utilisée                                           | Rarement nécessaire                    |
| Réduction dimensionnelle (PCA)              | Parfois               | Souvent inutile (arbre gère variables)                           | Souvent non nécessaire                 |
                                 



## **Transformations interactives :**

In [9]:
X['temp_precip'] = X['température'] * X['precipitation']
X['humid_temp'] = X['humidity'] * X['température']
X['precip_wind']=X['vitesse_vent']*X['precipitation']



In [10]:
# Transformer les features
X_prepared2 = preprocessor.fit_transform(X)

# Vérification :
print(X_prepared2.shape)


(43769, 58)


In [11]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_prepared2, y, test_size=0.2, random_state=42) # 20% test / 80% train

print(X_train2.shape, X_test2.shape)

(35015, 58) (8754, 58)


In [12]:

tree_model.fit(X_train2, y_train2)
y_pred_train2 = tree_model.predict(X_train2)
y_pred_test2 = tree_model.predict(X_test2)

print("R² Train :", r2_score(y_train2, y_pred_train2))
print("R² Test  :", r2_score(y_test2, y_pred_test2))
print("MSE Test :", mean_squared_error(y_test2, y_pred_test2))


R² Train : 0.7899012303992994
R² Test  : 0.755222071778963
MSE Test : 1.6781744593286196


In [13]:
X['temp_hum_precip']=X['température'] * X['humidity'] * X['precipitation']
X['temp_wind']=X['vitesse_vent']*X['température']
X['temp_wind_precip']=X['température'] * X['vitesse_vent'] * X['precipitation']
X['temp_weekend']=X['température'] * X['Weekend'] 
X['temp_ramadan']=X['température'] * X['Ramadan'] 
X['temp_newyear']=X['température'] * X['Nouvel An']
X['temp_jourferiés']=X['température'] * X['Jours Fériés'] 
X['temp_vacance']=X['température'] * X['Vacances Scolaires'] 
X['precip_weekend']=X['precipitation'] * X['Weekend'] 
X['precip_ramadan']=X['precipitation'] * X['Ramadan'] 
X['precip_newyear']=X['precipitation'] * X['Nouvel An']
X['precip_jourferiés']=X['precipitation'] * X['Jours Fériés'] 
X['precip_vacance']=X['precipitation'] * X['Vacances Scolaires'] 
X['vag_precip'] = X['precipitation'] * X['Indice de Pluie Intense']
X['humid_sech']=X['humidity'] * X['Indice de sécheresse']

In [63]:
# Transformer les features
X_prepared3 = preprocessor.fit_transform(X)

# Vérification :
print(X_prepared3.shape)


(43769, 73)


In [64]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_prepared3, y, test_size=0.2, random_state=42) # 20% test / 80% train

print(X_train3.shape, X_test3.shape)

(35015, 73) (8754, 73)


In [65]:

tree_model.fit(X_train3, y_train3)
y_pred_train3 = tree_model.predict(X_train3)
y_pred_test3 = tree_model.predict(X_test3)

print("R² Train :", r2_score(y_train3, y_pred_train3))
print("R² Test  :", r2_score(y_test3, y_pred_test3))
print("MSE Test :", mean_squared_error(y_test3, y_pred_test3))


R² Train : 0.7898686118288494
R² Test  : 0.7545448522101482
MSE Test : 1.6780463601002273


In [14]:
X=X.drop(columns=['temp_weekend','temp_ramadan','temp_newyear','temp_jourferiés',
'temp_vacance','precip_weekend','precip_ramadan','precip_newyear',
'precip_jourferiés','precip_vacance','vag_precip','humid_sech'])

In [68]:
# Transformer les features
X_prepared4 = preprocessor.fit_transform(X)

# Vérification :
print(X_prepared4.shape)
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_prepared4, y, test_size=0.2, random_state=42) # 20% test / 80% train

print(X_train4.shape, X_test4.shape)

tree_model.fit(X_train4, y_train4)
y_pred_train4 = tree_model.predict(X_train4)
y_pred_test4 = tree_model.predict(X_test4)

print("R² Train :", r2_score(y_train4, y_pred_train4))
print("R² Test  :", r2_score(y_test4, y_pred_test4))
print("MSE Test :", mean_squared_error(y_test4, y_pred_test4))



(43769, 61)
(35015, 61) (8754, 61)
R² Train : 0.7899423558076092
R² Test  : 0.7550233802656585
MSE Test : 1.6768038315754732


In [15]:
X=X.drop(columns=['temp_hum_precip','temp_wind','temp_wind_precip'])

In [16]:
# Transformer les features
X_prepared = preprocessor.fit_transform(X)

# Vérification :
print(X_prepared.shape)
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y, test_size=0.2, random_state=42) 

print(X_train.shape, X_test.shape)

tree_model.fit(X_train, y_train)
y_pred_train = tree_model.predict(X_train)
y_pred_test = tree_model.predict(X_test)

print(f"R² Train :{r2_score(y_train, y_pred_train):.4f}")
print(f"R² Test  : {r2_score(y_test, y_pred_test):.4f}")
print(f"MSE Test : {mean_squared_error(y_test, y_pred_test):.4f}")



(43769, 58)
(35015, 58) (8754, 58)
R² Train :0.7899
R² Test  : 0.7552
MSE Test : 1.6782


In [20]:
print(y_test.shape,y_pred_test.shape)

(8754, 13) (8754, 13)


**Afin d’améliorer les performances du modèle d’arbre de décision, plusieurs transformations interactives ont été testées. Ces transformations visaient à mieux capturer les relations non linéaires entre les variables indépendantes et la variable cible.**

**Après expérimentation, la meilleure configuration obtenue est la suivante :**

* **R² (Train) : 0.7899**

* **R² (Test) : 0.7552**

* **MSE (Test) : 1.6782**

> ### Ces résultats montrent que le modèle a pu mieux généraliser sur les données de test, tout en réduisant significativement l’erreur quadratique moyenne par rapport aux versions antérieures. 

In [74]:

# 1. Convertir y_pred en DataFrame si besoin
if isinstance(y_pred_test, np.ndarray):
    y_pred_test = pd.DataFrame(y_pred_test, columns=y_test.columns)

# 2. Concaténer les données
df_long = pd.DataFrame()
for col in y_test.columns:
    temp_df = pd.DataFrame({
        'product': [col] * len(y_test),
        'y_true': y_test[col].values,
        'y_pred': y_pred_test[col].values
    })
    df_long = pd.concat([df_long, temp_df], ignore_index=True)

# 3. Calculer les métriques
metrics = []
for col in y_test.columns:
    r2 = r2_score(y_test[col], y_pred_test[col])
    mse = mean_squared_error(y_test[col], y_pred_test[col])
    metrics.append({'product': col, 'R2': r2, 'MSE': mse})

df_metrics = pd.DataFrame(metrics)

# 4. Fusion finale
df_final = df_long.merge(df_metrics, on='product')


df_summary = df_final.groupby('product')[['R2', 'MSE']].first().reset_index()
print(df_summary)


                     product        R2       MSE
0                    Charbon  0.881188  0.782740
1           boissons chaudes  0.951853  1.530621
2          boissons fraiches  0.953166  0.911600
3              crème solaire  0.903259  0.686497
4   produits anti_moustiques  0.888749  3.060397
5      produits de jardinage  0.849348  0.659326
6    produits laitiers frais  0.785306  3.247335
7               snacks salés  0.582767  2.893299
8              snacks sucrés  0.742470  3.058964
9           soins hydratants  0.248250  0.825454
10             soins hygiene  0.781368  0.640814
11       ustensiles jetables  0.959884  2.215903
12     équipements d urgence  0.290279  1.303318


# **Cross Validation**

In [97]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

def evaluate_cv_model(x, y, cv=5, scoring='r2'):
 

    # Évaluation par cross-validation
    scores = cross_val_score(tree_model, x, y, cv=cv, scoring=scoring)

    # Affichage
    if scoring == 'r2':
        print(f"R² par fold : {np.round(scores, 4)}")
        print(f"Score R² moyen sur {cv} folds : {scores.mean():.4f}")
    elif scoring == 'neg_mean_squared_error':
        mse_scores = -scores
        print(f"MSE par fold : {np.round(mse_scores, 4)}")
        print(f"MSE moyen sur {cv} folds : {mse_scores.mean():.4f}")
    else:
        print("Scoring non reconnu")


evaluate_cv_model(X_train, y_train, cv=2, scoring='r2')
evaluate_cv_model(X_train, y_train,  cv=3, scoring='r2')
evaluate_cv_model(X_train, y_train, cv=4, scoring='r2')
evaluate_cv_model(X_train, y_train, cv=5, scoring='r2')
evaluate_cv_model(X_train, y_train,  cv=6, scoring='r2')
evaluate_cv_model(X_train, y_train,  cv=7, scoring='r2')
evaluate_cv_model(X_train, y_train,  cv=8, scoring='r2')
evaluate_cv_model(X_train, y_train, cv=9, scoring='r2')
evaluate_cv_model(X_train, y_train, cv=10, scoring='r2')




R² par fold : [0.7332 0.7358]
Score R² moyen sur 2 folds : 0.7345
R² par fold : [0.742  0.7409 0.744 ]
Score R² moyen sur 3 folds : 0.7423
R² par fold : [0.7438 0.7443 0.7482 0.745 ]
Score R² moyen sur 4 folds : 0.7453
R² par fold : [0.7457 0.7479 0.7455 0.7484 0.7457]
Score R² moyen sur 5 folds : 0.7466
R² par fold : [0.7453 0.7471 0.7436 0.7508 0.7494 0.7479]
Score R² moyen sur 6 folds : 0.7474
R² par fold : [0.7469 0.7462 0.7486 0.7463 0.7513 0.746  0.7496]
Score R² moyen sur 7 folds : 0.7478
R² par fold : [0.7474 0.7465 0.7504 0.7453 0.75   0.7532 0.7466 0.7494]
Score R² moyen sur 8 folds : 0.7486
R² par fold : [0.7462 0.7505 0.7505 0.7463 0.7456 0.7519 0.7533 0.7475 0.7497]
Score R² moyen sur 9 folds : 0.7491
R² par fold : [0.7483 0.7489 0.7495 0.749  0.7456 0.7522 0.7516 0.7496 0.7498 0.7497]
Score R² moyen sur 10 folds : 0.7494


> ### À 10 folds, le score est stabilisé autour de 0.749, ce qui est un bon indicateur de généralisation.

# **CONCLUSION**


Le modèle **DecisionTreeRegressor** a été entraîné et évalué sur notre jeu de données, offrant une approche intuitive et non linéaire pour prédire la variable cible.

Les résultats montrent que cet arbre de décision est capable de capturer des relations complexes entre les variables explicatives et la cible, avec un coefficient de détermination $R^2$ satisfaisant (0.7552). La validation croisée confirme la stabilité relative du modèle à travers différents sous-ensembles des données, bien que des variations légères soient observées selon le nombre de folds utilisés.

Cependant, plusieurs points peuvent être améliorés :

* L’arbre de décision tend à sur-ajuster les données d’entraînement si aucune régularisation ou limitation de profondeur n’est appliquée, ce qui pourrait nuire à sa généralisation.
* Le modèle pourrait bénéficier d’une optimisation des hyperparamètres (profondeur maximale, nombre minimal d’échantillons par feuille, etc.) pour trouver un meilleur compromis biais-variance.


En résumé, le **DecisionTreeRegressor** constitue une bonne première approche pour ce problème de régression, offrant à la fois simplicité et interprétabilité. Pour aller plus loin, l’exploration de modèles plus complexes et la mise en place d’une recherche d’hyperparamètres sont recommandées afin d’améliorer la précision et la robustesse des prédictions.

---
